# Assignment 1: Unit tests and coverage

- [1. Introduction](#1.-Introduction)
- [2. Coverage](#2.-Coverage)
    - [2.1 Statement coverage](#2.1-Statement-coverage)
    - [2.2 Branch coverage](#2.2-Branch-coverage)
    - [2.3 Dataflow coverage](#2.3-Dataflow-coverage)
- [3. More unit tests](#3.-More-unit-tests)
- [4. Mocking](#4.-Mocking)
- [5. Coverage revisited](#5.-Coverage-revisited)
- [BONUS: `doctest`](#BONUS:-doctest)
- [6. Submit to Canvas](#6.-Submit-to-Canvas)

## 1. Introduction

For a new self-driving car, we need an implementation of a high-precision pi: ChatGPT v4 suggests the following implementation for computing pi in Python, including a unit test. The code is packed in the two files `estimate_pi.py` and `test_estimate_pi.py`. 

Run the existing test using your shell (every cell starting with an `!` will be executed in your OS's shell). 

In [6]:
!python3 -m pytest test_estimate_pi.py

============================= test session starts ==============================
platform darwin -- Python 3.9.7, pytest-7.1.2, pluggy-1.0.0
rootdir: /Users/mariusstokkedal/Desktop/PA1465_Test_av_mjukvara/Assignment_1
plugins: anyio-3.5.0, mock-3.7.0, cov-3.0.0
collected 2 items                                                              

test_estimate_pi.py ..                                                   [100%]

============================== 2 passed in 2.40s ===============================


What is problematic with that test ChatGPT created for us and would you address this problem?

The test function for the function `estimate_pi` is only tested with one valid value. If a value less than 0 is sent as argument it will return **π = 0** which is undeniably incorrect; and if 0 is inserted the program will crash when deviding by 0 (`return 4 * count / n`). Since this function is meant to be located in a self driving car errors could have devastating consequences. If **π** is estimated to 0, calculating for example speed based on wheel rotation speed will always be **0**.  Further more, *delta* should be smaller than 0.01 because a **high precission** is requested. Lastly, the main function in `estimate_py.py`is not tested, but I assume this is not the objective of the test.

***
## 2. Coverage

### 2.1 Statement coverage
Compute the statement coverage of the program using [`coverage.py`](https://coverage.readthedocs.io/en/latest/index.html). 

In [28]:
!coverage run -m OG_test_estimate_pi
!coverage report -m

.
----------------------------------------------------------------------
Ran 1 test in 2.014s

OK
Name                     Stmts   Miss  Cover   Missing
------------------------------------------------------
OG_estimate_pi.py           26     12    54%   18-19, 23-36
OG_test_estimate_pi.py       9      0   100%
------------------------------------------------------
TOTAL                       35     12    66%


How can we interprete the results?

The report tells us that 54% of the statements are executed in`estimate_pi.py` and 100% in `test_estimate_pi.py`. However, the lines marked as "Missing" are the main code for both of the files. This code is only run if the code is run directly by a command and not being imported to another file. Since the files are run from covareage and not directly the main code will not be running and is therefore marked as missing. The lines 18-19 in `estimate_pi.py`are located in the method `write`from the class `PiFileWriter`, this is not called from the test file and therefore not run.

With this in mind the coverage of the method `estimate_pi`is 100% but the `PiFileWriter`-class is not called and the code isn't run as the main file resulting in the main code not being run.

### 2.2 Branch coverage
Now compute the statement coverage of the program using [`coverage.py`](https://coverage.readthedocs.io/en/latest/index.html). 

In [29]:
!coverage run --branch OG_test_estimate_pi.py
!coverage report -m

.
----------------------------------------------------------------------
Ran 1 test in 2.191s

OK
Name                     Stmts   Miss Branch BrPart  Cover   Missing
--------------------------------------------------------------------
OG_estimate_pi.py           26     12     10      1    53%   18-19, 23-36
OG_test_estimate_pi.py       9      0      2      1    91%   11->exit
--------------------------------------------------------------------
TOTAL                       35     12     12      2    62%


How can we interprete the results?

The missing lines are the same as in 2.1 and the branch coverage can therefor be interpreted as 100% since we only want to test the function `estimate_pi`. The column *BrPart* is short for *Branch Partial* and refers to `if __name__ == '__main__':`, this is partial since it is always true if the code is main and always false if it is imported.

### 2.3 Dataflow coverage

Draw the flow graph for the function `estimate_pi` defined in `estimate_pi.py`. Annotate the graph with definition and use information. Note: Please submit a separate image file or PDF with the name `dataflow_coverage.<file_extension>` for this task.

Identify and describe the minimum number of test cases to achieve: all-defs coverage, and all-uses coverage. 

#### All-defs
To cover all defs of all variables in `estimate_pi` the shortest possible path is the nodes: **1, 2, 3, 4, 5, 2, 6**. Node *4* is a if-statement depending on the result from node *3* which is a random value. To ensure node *5* is executed **one** test case is enoug assuming the argument *N* is large enough, e.g. **1000**, to ensure that node *4's* condition is true at least once.

#### All-uses
In the case of *all-use* the result is almost the same as in *all-defs* but it requires all possible outcomes of a if-statement or conditional for-loop to be covered. In this cas we have both. The shortest possible path for *all-use* is therefore **1, 2, 3, 4, 2, 3, 4, 5, 2, 6** alternativly **1, 2, 3, 4, 5, 2, 3, 4, 2, 6**

***
## 3. More unit tests

Add two more unit tests with the principles you learned in the lecture. Describe what principle you have used.

In [6]:
def test_estimate_pi_illigal_argument(self):
    with self.assertRaises(ValueError):
        estimate_pi(-1)
            
def test_estimate_pi_wrong_data_type():
    with self.assertRaises(TypeError):
            estimate_pi('test')

## F.I.R.S.T
**F.I.R.S.T** is short for Fast, Isolated, Repeateble, Self-validating and Thorough

### F – Fast
Both tests will have finite running time and is therefore considered as fast in this case.

### I – Isolated
Both tests are isolated since they only need them selves and the function `estimate_pi` to be executed.

### R – Repeateble
Both test are independent from the machine they are executed on, considering *operating system, software and hardware*. A test of `PiFileWriter` would be hard to make repeateble since the disk may be accesed differently on other operating ssystems. Stationary values are also used, i.e. they are not for example generated by a random value

### S – Self-validating
Both tests are self-validating since we know what the desired output is and therefore they also achives this criteria.

### T – Thorough
As discussed earlier, the original test lacks a test for invalid values. With these two tests we test with an input that is too small and another which is of an invalid type (string instead of int).

***
## 4. Mocking

We want to store the resulting number persistently on our file system. We use the following class. 

In [12]:
class PiFileWriter:
    @staticmethod
    def write(content, file_path):
        with open(file_path, 'w') as file:
            file.write(content)

Implement a test double for `PiFileWriter` and add your implementation to `test_estimate_pi.py`. Discuss what type of test double you have implemented.

I replaced the class `PiFileWriter`with this:

In [14]:
class MockPiFileWriter:
    def __init__(self):
        self._content = None
        self._file_path = None

    def write(self, content, file_path):
        self._content = content
        self._file_path = file_path

    def content(self):
        return self._content

    def file_path(self):
        return self._file_path

And then added this method to `test_estimate_pi.py`:

In [ ]:
def testPiFileWriter(self):
    mock_file_writer = MockPiFileWriter()
    estimate = estimate_pi(1000000)
    path = '/test.txt'
    mock_file_writer.write(estimate, path)
    self.assertEqual(mock_file_writer.content(), estimate)
    self.assertEqual(mock_file_writer.file_path(), path)

This mock saves the *content* and the *file path* as variables in the instance of the class instead of acctually writing the content to the given adress on the disk. This makes it possible to run the same test code on multiple different operative systems without the need of modifications.

**Name three other types of unit tests you would want to mock and explain why.**

### 1.
Since `estimate_pi` relies on random values for the variables *x* and *y* it can be hard to determine which values they will take. Therefore `random.uniform(-1, 1)` should be mocked with a function that behaves like `random`, but also generates predicteble numbers. This will result in `estimate_pi` being easier to test since its result will always be reproduceble and the test will not have to relie on any external code.

### 2.
In a more general scope database access should be mocked. Relying on a databaase during testing can both increase the run time of the test and affect the data it self by potentially producing inconsistencies or corrupt data. If the database instead is mocked, we can simulate the data and test the code's behavior and performance witout relying on a database and its data behaving as we want.

### 3.
Lastly, time-related functions is important to mock during testing since their output is affected by, as the name suggests, the current time for the test. By mocking the time-related functions, we can simulate different time scenarios and test the behavior of our code. This is particularly important for testing time-sensitive logic, such as expiration dates or time-based calculations.

***
## 5. Coverage revisited

Rerun statement and branch coverage and discuss the differences and changes.

In [34]:
!coverage run -m test_estimate_pi
!coverage report -m

..
----------------------------------------------------------------------
Ran 2 tests in 4.393s

OK
Name                  Stmts   Miss  Cover   Missing
---------------------------------------------------
estimate_pi.py           26     12    54%   18-19, 23-36
test_estimate_pi.py      28      0   100%
---------------------------------------------------
TOTAL                    54     12    78%


In [35]:
!coverage run --branch test_estimate_pi.py
!coverage report -m

..
----------------------------------------------------------------------
Ran 2 tests in 4.523s

OK
Name                  Stmts   Miss Branch BrPart  Cover   Missing
-----------------------------------------------------------------
estimate_pi.py           26     12     10      1    53%   18-19, 23-36
test_estimate_pi.py      28      0      2      1    97%   37->exit
-----------------------------------------------------------------
TOTAL                    54     12     12      2    73%


The percentage for *cover* went up for `test_estimate_pi.py`in the branch-test. This is because the ratio of code being tested versus not tested went up resulting in a higher coverage. The run time on the other hand has now almost dubbeled, goning from *~2 seconds* to *~4.5 seconds* which is not much in absolute numbers but it is still over *50%* ans will result in big increasments in time if the function is tested multiple times.

***
# BONUS: `doctest`

If you are curious or want to stand out, check out [`doctest`](https://en.wikipedia.org/wiki/Doctest). This task is optional. 

Add two `doctest` test cases and run the `doctest` tests.

In [30]:
!python3 test_estimate_pi.py

TestResults(failed=0, attempted=3)
...
----------------------------------------------------------------------
Ran 3 tests in 1.996s

OK


How do you like `doctest`?

I like it since it gives a better understanding for a developer who has not been involved in the development of the function because you can directly see what result is expected from differnet inputs without the need for manual calculation or tests. But it is limited to cases where you know what result will be produced, if the function generates a random result without the possebility to predict it it is hard to write a `doctest` for that input.

## 6. Submit to Canvas

Almost done, but the most tricky part is missing: submitting. :)

Before submitting, make sure
- you completed all non-optional tasks in this assignment (i.e., all empty cells are filled with meaningful content)
- you don't use external libraries except `coverage.py`
- the notebook runs straight through
- your test code works
- your code is readable and follows the Python coding conventions

All set? Great. Just two steps away from happiness. 

1. Go through the list above and check again
2. Submit *three* files to canvas:
    - `assignment.ipynb`
    - `test_estimate_pi.py`
    - `dataflow_coverage.<file_extension>`
3. Take a deep breath and carpe diem.
